In [3]:
import pandas as pd, numpy as np
import re
import time

### 1. 가게리스트_reviews.csv 파일 불러오기
- storeid : 가게id
- Title : 공공데이터포털에 등록된 가게명
- storename : Title에서 공백을 제거함
- search_area : 
    - Sigungu가 2글자 이하일 경우(ex.광주 동구,서울 중구) -> Region (ex.광주,서울)
    - Sigungu가 2글자보다 많을 경우(ex.서울 종로구,경기도 수원시) -> Sigungu (ex.종로,수원)
- Category1 : 한식,중국식,일식,서양식,아시아식,기타외국음식 - 음식점 대분류
- Category2 : 냉면/쌀국수/치킨 등 음식점 소분류
- menu : 떡갈비,비빔밥/갈비살,안창살/장어구이 등 가게의 (대표)메뉴
    - Category2와 menu는 겹쳐도 됨, 둘 중 한 컬럼만 있어도 됨
- Blog : 가게명으로 네이버 상세검색 했을 때 나온 블로그 리뷰의 개수
- reviews_num : 정제한 리뷰 데이터 79만개 중, 해당 가게의 리뷰의 개수

In [15]:
stores = pd.read_csv('./train/data/가게리스트_reviews.csv', encoding='utf-8')
stores.drop('Unnamed: 0', axis=1, inplace=True)
stores.shape

(22782, 11)

In [17]:
# 가게는 총 22,782개
stores.head(3)

,storeid,Title,storename,Region,Sigungu,search_area,Category1,Category2,menu,Blog,reviews_num
0,13992,빛고을떡갈비,빛고을떡갈비,광주,광산구,광산,한식,"떡갈비,비빔밥",NaN,24,2
1,13995,착한소장수,착한소장수,광주,동구,광주,NaN,"갈비살, 안창살",NaN,4155,7
2,14000,고흥나루터,고흥나루터,광주,동구,광주,NaN,장어구이,NaN,116,3


In [113]:
# 여러명이 작업하는 경우 나중에 통합을 쉽게 하기 위해서 인덱스를 storeid로 부여한다.
# storeid는 각 가게당 고유한 값을 갖는 id이다.
stores.set_index('storeid',inplace=True)
stores.head(3)

,Title,storename,Region,Sigungu,search_area,Category1,Category2,menu,Blog,reviews_num
storeid,,,,,,,,,,
13992,빛고을떡갈비,빛고을떡갈비,광주,광산구,광산,한식,"떡갈비,비빔밥","떡갈비,비빔밥",24,2
13995,착한소장수,착한소장수,광주,동구,광주,NaN,"갈비살, 안창살","갈비살, 안창살",4155,7
14000,고흥나루터,고흥나루터,광주,동구,광주,NaN,장어구이,장어구이,116,3


In [115]:
# 가게 리스트가 총 22,782개이므로 index의 unique값도 22,782이다.
stores.index.unique()

Int64Index([ 13992,  13995,  14000,  14001,  14002,  14005,  14006,  14008,
             14010,  14012,
            ...
            173613, 167542, 172875, 180834, 154810, 151855, 130419, 113445,
            164368, 140758],
           dtype='int64', name='storeid', length=22782)

### 2. 챗봇을 위한 가게리스트 데이터 전처리
- Category1 음식점 대분류(한식,중식,일식,서양식,기타아시아식,기타외국음식) 통일 (nan 있으면 안됨)
- Category2 음식점 소분류(분식,경양식,치킨,횟집,뷔페,패밀리레스토랑,고기구이,푸드트럭 등등) 최대한 nan 제거
- menu 음식점 대표메뉴 (떡갈비,비빔밥,갈비살,안창살 등등) 쉼표로 구분하여 한 가게 당 여러 개 있어도 됨

#### 2-1. 먼저 Category1, Category2, menu 컬럼을 살펴보자!

In [20]:
# 현재 menu 컬럼은 값이 없다.
stores.menu.unique()

array([nan])

In [6]:
# Category1 컬럼
stores.Category1.unique()

array(['한식', nan, '경양식', '기타', '김밥(도시락)', '분식', '뷔페식', '식육(숯불구이)', '일식',
       '중국식', '통닭(치킨)', '회집', '양식', '식육', '중식', '일반음식점', '탕류(보신용)',
       '패스트푸드', '외국음식전문점(인도,태국등)', '외국음식전문점', '푸드트럭', '이색음식점', '모범음식점',
       '복어취급', '아시아식', '뷔페', '냉면집', '패밀리레스트랑', '횟집', '채식전문점', '서양식',
       '분식점', '음식(한식)', '찜갈비', '별식', '패밀리레스토랑', '라이브카페'], dtype=object)

In [18]:
# Category2 컬럼은 현재 음식 메뉴가 들어가있다.
stores.Category2.unique()

array(['떡갈비,비빔밥', '갈비살, 안창살', '장어구이', '통오리구이', '복탕', '비빔밥', '돼지갈비', '짜장면',
       '콩나물국밥', '순두부찌개', '민물장어구이', nan, '김치찌개', '장어탕', '삼겹살', '자장면', '커트',
       '민어', '생선찜', '생선회', '아구탕', '생선회,찜', '전복', '활어회', '갈매기살', '꽃게탕',
       '꽃게, 돌게정식', '묵은지감자탕', '생태탕', '오리불고기', '아귀찜,탕', '삼계탕', '생선구이',
       '함박스테이크', '일식', '게장', '갈비탕', '차돌박이갓삼합', '꽃게게장정식', '머리국밥,머리수육',
       '꼬막', '국밥', '염소고기', '백반', '회', '도토리요리', '한정식', '낙지요리', '샤브샤브',
       '닭요리', '대구찜', '떡갈비', '소고기', '산낙지비빔밥', '장어요리', '양식', '쌈밥', '냉면',
       '낙지', '곱창', '영양솥밥', '감자탕', '장어', '복탕,복찜,아구찜', '장어구이,장어탕',
       '한정식,매생이국', '홍어정식,삼합,보리애국', '한정식,코스요리,굴비정식', '흑염소탕, 전골', '떡갈비  정식',
       '쌈밥정식', '죽순나물밥 정식', '대통밥', '보쌈, 부대찌개', '능이닭곰탕', '매운탕',
       '한우구이, 삼겹살구이', '대구탕, 장어구이', '쏘가리탕', '활어회, 매운탕, 된장물회',
       '돌솥밥, 오리로스구이, 오리탕', '활어회, 바지락회, 키조개구이', '등심,꽃살구이,갈비탕,육개장', '한식뷔페',
       '생선회,매운탕', '떡갈비정식', '토종닭', '쌈밥/오삼불고기/오리.삼겹살.닭구이',
       '돼지국밥/뼈해장국/삼겹살수육', '삼겹살/오리구이.탕/돼지갈비.찜/백반',
       '한정식/영양돌솥밥/고등어쌈밥정식/약선육낙전골/육낙', '묵은지찜', '꽃게무침', '

In [21]:
# Category2 컬럼을 복붙하여 menu로 옮긴다.
stores['menu'] = stores['Category2']

In [116]:
# 복붙이 잘 되었는지 확인
stores.head(3)

,Title,storename,Region,Sigungu,search_area,Category1,Category2,menu,Blog,reviews_num
storeid,,,,,,,,,,
13992,빛고을떡갈비,빛고을떡갈비,광주,광산구,광산,한식,"떡갈비,비빔밥","떡갈비,비빔밥",24,2
13995,착한소장수,착한소장수,광주,동구,광주,NaN,"갈비살, 안창살","갈비살, 안창살",4155,7
14000,고흥나루터,고흥나루터,광주,동구,광주,NaN,장어구이,장어구이,116,3


#### 2-2. Category1 컬럼 정제
- 한식,중국식,일식,서양식,아시아식,기타외국음식 
- 6개로 대분류한다.

In [23]:
# na 값을 살펴보자. na는 Category2와 menu의 정제가 어느 정도 완료되면 그 때 채우도록 한다.
stores[stores.Category1.isna()].shape

(2194, 11)

In [25]:
# Category1의 종류가 현재 몇 개 있는지 살펴보자!
stores.groupby('Category1').count().storeid.sort_values(ascending=False)

Category1
한식                 12542
일반음식점               1635
분식                  1505
식육(숯불구이)            1124
경양식                  933
일식                   849
회집                   368
기타                   250
중국식                  220
뷔페식                  182
통닭(치킨)               173
탕류(보신용)              115
양식                   105
중식                   104
외국음식전문점(인도,태국등)       91
김밥(도시락)               86
서양식                   73
패스트푸드                 63
냉면집                   27
푸드트럭                  20
아시아식                  16
복어취급                  13
외국음식전문점               13
뷔페                    12
분식점                   12
모범음식점                 12
패밀리레스트랑               10
별식                     8
이색음식점                  7
채식전문점                  7
패밀리레스토랑                5
식육                     2
찜갈비                    2
라이브카페                  2
음식(한식)                 1
횟집                     1
Name: storeid, dtype: int64

#### 1)

In [40]:
# '중식' -> '중국식'으로 통일
stores.loc[stores[stores.Category1=='중식'].index,'Category1'] = '중국식'
# '음식(한식)' -> '한식'
stores.loc[stores[stores.Category1=='음식(한식)'].index,'Category1'] = '한식'
# '양식' -> '서양식'
stores.loc[stores[stores.Category1=='양식'].index,'Category1'] = '서양식'

In [41]:
stores.Category1.unique()

array(['한식', nan, '경양식', '기타', '김밥(도시락)', '분식', '뷔페식', '식육(숯불구이)', '일식',
       '중국식', '통닭(치킨)', '회집', '서양식', '식육', '일반음식점', '탕류(보신용)', '패스트푸드',
       '외국음식전문점(인도,태국등)', '외국음식전문점', '푸드트럭', '이색음식점', '모범음식점', '복어취급',
       '아시아식', '뷔페', '냉면집', '패밀리레스트랑', '횟집', '채식전문점', '분식점', '찜갈비', '별식',
       '패밀리레스토랑', '라이브카페'], dtype=object)

##### 2) Category2로 이동할 값들 찾기
- 경양식, 김밥(도시락), 분식, 뷔페식, 식육(숯불구이) 등등 
- 위와 같은 소분류는 모두 Category2로 복붙하고 Category1은 '한식'으로 분류한다.
- Category2에 있던 값이 사라지면 어떻게 될지 걱정하지 않아도 된다.
- Category2는 menu와 현재 동일한 값을 가지고 있으니 상관 없다.

In [117]:
# Category1이 '경양식'으로 분류된 애들은 'Category2' 값으로 '경양식'을 주고 'Category1'은 '한식'으로 바꾼다.
stores[stores.Category1 == '경양식'].head(3)

,Title,storename,Region,Sigungu,search_area,Category1,Category2,menu,Blog,reviews_num
storeid,,,,,,,,,,


In [46]:
# stores.loc[바꿀 행의 인덱스, 바꿀 컬럼] = 바꿀 값

# 먼저 Category2 값을 바꾼다.
stores.loc[stores[stores.Category1=='경양식'].index,'Category2'] = '경양식'
# 그 다음에 Category1 값을 바꾼다.
stores.loc[stores[stores.Category1=='경양식'].index, 'Category1'] = '한식'

In [47]:
stores.Category1.unique()

array(['한식', nan, '기타', '김밥(도시락)', '분식', '뷔페식', '식육(숯불구이)', '일식', '중국식',
       '통닭(치킨)', '회집', '서양식', '식육', '일반음식점', '탕류(보신용)', '패스트푸드',
       '외국음식전문점(인도,태국등)', '외국음식전문점', '푸드트럭', '이색음식점', '모범음식점', '복어취급',
       '아시아식', '뷔페', '냉면집', '패밀리레스트랑', '횟집', '채식전문점', '분식점', '찜갈비', '별식',
       '패밀리레스토랑', '라이브카페'], dtype=object)

In [59]:
# 경양식이 정리되었다.
# 같은 방식으로 다른 모든 Category1을 대분류 6가지로 나눈다. (nan 값 제외)
# 대분류 : 한식,중국식,일식,서양식,아시아식,기타외국음식  (6개) *추가할 필요가 있으면 회의해서 추가하세요!
# 일반음식점,모범음식점 같이 애매한 건 그냥 '한식'으로 분류해도 상관없다.

<br><br>

#### 2-3. 가게이름으로 'Category2' 채우기 

In [102]:
# re 사용법 익히기
# re.search(키워드,검색할스트링) -> '검색할스트링'에서 '키워드'를 찾는다.
#                           -> '키워드'를 발견하면 첫 번째 발견한 키워드의 위치를 반환한다.
#                           ->  첫 번째 키워드를 발견했다면 즉시 re.search 메소드가 종료된다. 두 번째 키워드까지 찾지 않는다는 뜻.
#                           -> '키워드'를 발견하지 못하면 None을 반환한다.

ss='오리고기 돼지고기 양고기'
rr = re.search('고기고기',ss)
print(rr)
rr = re.search('고기',ss)
print(rr)

None
<_sre.SRE_Match object; span=(2, 4), match='고기'>


In [106]:
# 이 메소드를 Series에 사용하려면? Series.str.find 메소드를 써야한다. 
# (Series.str : 시리즈안의 값이 str일 때 스트링메소드를 적용하기 위해 쓰는 메소드)
dd = pd.DataFrame({
    'ss' : ['오리고기', '양고기', '스타벅스', '돼지고기', '냉면', '양장피', '맛있는고기', '고기전문점']
})

# Series.str.find(키워드) 메소드는 키워드를 발견하면 그 키워드의 시작 위치를 반환한다.
# 키워드를 발견하지 못하면 -1을 반환한다.
dd['find'] = dd.ss.str.find('고기')
dd

,ss,find
0,오리고기,2
1,양고기,1
2,스타벅스,-1
3,돼지고기,2
4,냉면,-1
5,양장피,-1
6,맛있는고기,3
7,고기전문점,0


In [105]:
# 데이터프레임의 ss컬럼에서 '고기'를 포함한 결과만 추출하고 싶다면?
# '불리언 색인'과 'Series.str.find' 메소드를 이용한다.
dd[dd['ss'].str.find('고기') != -1]

,ss,find
0,오리고기,2
1,양고기,1
3,돼지고기,2
6,맛있는고기,3
7,고기전문점,0


In [63]:
# Category2 na값을 살펴보자. 엄청 많다.. 이걸 웬만하면 다 채우는 것이 좋다.
# Category2는 매우 중요하다 '경양식'집 추천해줘 할 때 Category2에 '경양식' 값이 들어있어야 검색이 되어서 추천이 가능하기 때문이다.
stores[stores.Category2.isna()].shape

(19538, 11)

In [118]:
# 가게명에 '냉면'이 들어간 집은 냉면 전문점일 것이다.
keyword = '냉면'
stores[stores.Title.str.find(keyword) != -1]

,Title,storename,Region,Sigungu,search_area,Category1,Category2,menu,Blog,reviews_num
storeid,,,,,,,,,,
14664,오대옥천냉면,오대옥천냉면,전라남도,순천시,순천,NaN,냉면,냉면,15,4
16083,설악냉면,설악냉면,전라북도,부안군,부안,한식,냉면,"냉면, 콩나물국밥",97,2
17035,청수골냉면,청수골냉면,전라북도,임실군,임실,NaN,냉면,NaN,17,6
121055,(주)팔당냉면,(주)팔당냉면,경기도,양주시,양주,한식,냉면,NaN,50,1
144896,30년전통 속초코다리 냉면,30년전통속초코다리냉면,부산,사상구,사상,NaN,냉면,NaN,23,4
...,...,...,...,...,...,...,...,...,...,...
120993,홍경백함흥냉면,홍경백함흥냉면,경기도,안양시,안양,분식,냉면,NaN,16,6
140458,화성냉면,화성냉면,경기도,화성시,화성,한식,냉면,NaN,22,3
95444,화정함흥냉면,화정함흥냉면,경기도,고양시,고양,한식,냉면,한식,63,13


In [108]:
# 위 135개 row의 Category2에 '냉면'을 입력하면 된다!
# '설악냉면'의 Category2에는 '냉면, 콩나물국밥' 이렇게 2개가 있지만,
# '콩나물국밥'은 '설악냉면'의 menu에 이미 있기 때문에 Category2에서 지워도 상관없다.
# Category2는 이 가게의 정체성이라고 할 수 있는 '냉면'만 입력하는 것이 바람직하다.

# 데이터프레임 속에서 특정 row를 추출하고 그 row의 특정 컬럼 값을 바꾸고 싶을 때 쓰는 방법:
# 데이터프레임.loc[특정row의 index, 특정컬럼명] = 바꿀값

keyword = '냉면'
# Title에서 keyword인 '냉면'을 포함하는 row들의 index를 모두 idxs에 저장한다.
idxs = stores[stores.Title.str.find(keyword) != -1].index
stores.loc[idxs, 'Category2'] = '냉면'

In [119]:
# 다시 설악냉면이 잘 바꿔졌는지 살펴본다.
stores[stores.Title=='설악냉면']

,Title,storename,Region,Sigungu,search_area,Category1,Category2,menu,Blog,reviews_num
storeid,,,,,,,,,,
16083,설악냉면,설악냉면,전라북도,부안군,부안,한식,냉면,"냉면, 콩나물국밥",97,2


In [110]:
# 이런식으로 최대한 Category2를 채운다.
# 하다가 다시 되돌려야할 때도 있으니까 중간중간 저장을 해준다.
stores.to_csv('./가게리스트_reviews_정제중.csv', encoding='utf-8')

In [121]:
# Category2는 Category1처럼 완벽하게 정제할 필요는 없다.
# 예를 들면 Category2에 '떡갈비, 비빔밥'이 있을 경우, 챗봇에서 '떡갈비'를 검색할 때 '떡갈비, 비빔밥'도 검색결과에 포함되기 때문에,
# 굳이 비빔밥을 없앨 필요는 없다는 뜻이다. (하지만 떡갈비가 대표메뉴라면 없애는 것이 낫겠다.)
# 웬만하면 통일할 수 있는 것은 통일하는 것이 좋다.
# 예를 들면 '갈비살등','갈비','갈비류','갈비정식' -> '갈비'로 통일하는 것이 좋다.
# '갈비탕'은 갈비요리중에서도 특정한 요리이니까 그대로 냅둔다.
# '냉면','냉면집'은 -> '냉면'으로 통일한다.
stores.Category2.sort_values().unique()

array(['갈매기살', '갈비', '갈비. 삼겹살, 냉면', '갈비류', '갈비살', '갈비살, 안창살', '갈비살등',
       '갈비정식', '갈비탕', '갈치요리', '갈치조림', '갈치조림. 고등어조림', '감성주점', '감자탕',
       '게국지, 간장게장, 꽃게탕', '게장', '경양식', '고기', '고기국수. 비빔국수', '고기류', '고기뷔페',
       '고등어조림', '곤드레밥', '곰탕', '곰탕,설렁탕', '곱창', '곱창 등', '곱창구이', '곱창전골',
       '구이(쇠고기/삼겹살/돼지갈비)', '구이류, 버섯전골', '구이류,국밥', '국밥', '국수', '굴밥,굴요리',
       '굴비정식', '굴비한정식', '굴요리전문', '궁보계정, 북경식탕수육', '기타', '김밥(도시락)', '김치찌개',
       '까페', '꼬리곰탕', '꼬막', '꽃게, 돌게정식', '꽃게게장정식', '꽃게무침', '꽃게장', '꽃게탕',
       '낙지', '낙지요리', '남도요리', '냉면', '냉면집', '능이닭곰탕', '능이백숙코스',
       '능이오리백숙, 자연산버섯찌개, 김치찌개, 된장찌개', '다슬기탕, 국밥', '닭백숙', '닭요리', '닭한마리',
       '대구찜', '대구탕', '대구탕, 장어구이', '대방어회. 갈치조림', '대통밥', '더덕정식',
       '도루묵,가자미찌개', '도토리요리', '돈까스', '돈까스, 초계국수', '돌솥밥',
       '돌솥밥, 오리로스구이, 오리탕', '동태찜,해물찜', '동태탕', '돼지갈비', '돼지갈비, 삼겹살',
       '돼지갈비, 정식', '돼지갈비,돌솥밥', '돼지갈비,소갈비', '돼지갈비구이', '돼지고기',
       '돼지고기, 주꾸미, 냉면', '돼지고기정식', '돼지국밥', '돼지국밥/뼈해장국/삼겹살수육',
       '돼지막창, 양념돼지갈비, 삼겹살', '돼지불고기', '두루치기정식,육회비빔밥', '두부', '등심

In [ ]:
# 파이팅!